In [62]:
import keras
from keras.utils.np_utils import to_categorical

import pandas as pd
import numpy as np

In [4]:
data = pd.read_csv('./DATA/sensorless_data.csv')
data.head()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F40,F41,F42,F43,F44,F45,F46,F47,F48,TARGET
0,-3.014600e-07,8.260300e-06,-0.000012,-0.000002,-1.438600e-06,-0.000021,0.031718,0.031710,0.031721,-0.032963,...,-0.63308,2.9646,8.1198,-1.4961,-1.4961,-1.4961,-1.4996,-1.4996,-1.4996,1
1,2.913200e-06,-5.247700e-06,0.000003,-0.000006,2.778900e-06,-0.000004,0.030804,0.030810,0.030806,-0.033520,...,-0.59314,7.6252,6.1690,-1.4967,-1.4967,-1.4967,-1.5005,-1.5005,-1.5005,1
2,-2.951700e-06,-3.184000e-06,-0.000016,-0.000001,-1.575300e-06,0.000017,0.032877,0.032880,0.032896,-0.029834,...,-0.63252,2.7784,5.3017,-1.4983,-1.4983,-1.4982,-1.4985,-1.4985,-1.4985,1
3,-1.322600e-06,8.820100e-06,-0.000016,-0.000005,-7.282900e-07,0.000004,0.029410,0.029401,0.029417,-0.030156,...,-0.62289,6.5534,6.2606,-1.4963,-1.4963,-1.4963,-1.4975,-1.4975,-1.4976,1
4,-6.836600e-08,5.666300e-07,-0.000026,-0.000006,-7.940600e-07,0.000013,0.030119,0.030119,0.030145,-0.031393,...,-0.63010,4.5155,9.5231,-1.4958,-1.4958,-1.4958,-1.4959,-1.4959,-1.4959,1


### Format input data

In [10]:
x_cols = list(data.columns[:-1])
X_data = data[x_cols].copy()
# Adding 0 for easy reshaping
X_data['F49'] = 0
X_data.head()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F40,F41,F42,F43,F44,F45,F46,F47,F48,F49
0,-3.014600e-07,8.260300e-06,-0.000012,-0.000002,-1.438600e-06,-0.000021,0.031718,0.031710,0.031721,-0.032963,...,-0.63308,2.9646,8.1198,-1.4961,-1.4961,-1.4961,-1.4996,-1.4996,-1.4996,0
1,2.913200e-06,-5.247700e-06,0.000003,-0.000006,2.778900e-06,-0.000004,0.030804,0.030810,0.030806,-0.033520,...,-0.59314,7.6252,6.1690,-1.4967,-1.4967,-1.4967,-1.5005,-1.5005,-1.5005,0
2,-2.951700e-06,-3.184000e-06,-0.000016,-0.000001,-1.575300e-06,0.000017,0.032877,0.032880,0.032896,-0.029834,...,-0.63252,2.7784,5.3017,-1.4983,-1.4983,-1.4982,-1.4985,-1.4985,-1.4985,0
3,-1.322600e-06,8.820100e-06,-0.000016,-0.000005,-7.282900e-07,0.000004,0.029410,0.029401,0.029417,-0.030156,...,-0.62289,6.5534,6.2606,-1.4963,-1.4963,-1.4963,-1.4975,-1.4975,-1.4976,0
4,-6.836600e-08,5.666300e-07,-0.000026,-0.000006,-7.940600e-07,0.000013,0.030119,0.030119,0.030145,-0.031393,...,-0.63010,4.5155,9.5231,-1.4958,-1.4958,-1.4958,-1.4959,-1.4959,-1.4959,0


In [13]:
len(X_data)

58509

In [40]:
X = X_data.to_numpy().reshape((len(X_data),7,7,1))
X.shape

(58509, 7, 7, 1)

In [41]:
X[0].shape

(7, 7, 1)

In [75]:
data['TARGET'] = data['TARGET'] - 1
y = data['TARGET'].to_numpy()

In [76]:
y = to_categorical(y, num_classes=data['TARGET'].nunique())

### Randomizing dataset

In [77]:
idxs = np.arange(len(X))
samples = np.random.choice(idxs,size=10000)

X_rand = X[samples]
y_rand = y[samples]

### CNN Model

In [99]:
model = keras.models.Sequential([
    keras.layers.Conv2D(256, (2,2), strides=1, activation='relu', input_shape=(7,7,1)),
    keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    keras.layers.Conv2D(256, (2,2), strides=1, activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(11, activation='softmax')
])

In [100]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 6, 6, 256)         1280      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 3, 3, 256)         0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 2, 2, 256)         262400    
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 1, 1, 256)         0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 512)               131584    
_________________________________________________________________
dense_11 (Dense)             (None, 11)               

In [96]:
y_rand[:25000][0].shape

(11,)

In [98]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
history = model.fit(X_rand , y_rand, steps_per_epoch=1000, epochs = 10)

Epoch 1/10
1000/1000 [==============================] - 7s 7ms/step - loss: 0.5605 - accuracy: 0.7556
Epoch 2/10
1000/1000 [==============================] - 7s 7ms/step - loss: 0.5277 - accuracy: 0.7699
Epoch 3/10
1000/1000 [==============================] - 7s 7ms/step - loss: 0.5241 - accuracy: 0.7728
Epoch 4/10
1000/1000 [==============================] - 9s 9ms/step - loss: 0.4918 - accuracy: 0.7796
Epoch 5/10
1000/1000 [==============================] - 7s 7ms/step - loss: 0.4734 - accuracy: 0.7860
Epoch 6/10
1000/1000 [==============================] - 4s 4ms/step - loss: 0.4458 - accuracy: 0.8009
Epoch 7/10
1000/1000 [==============================] - 4s 4ms/step - loss: 0.4557 - accuracy: 0.8034
Epoch 8/10
1000/1000 [==============================] - 4s 4ms/step - loss: 0.4243 - accuracy: 0.8059
Epoch 9/10
1000/1000 [==============================] - 4s 4ms/step - loss: 0.4362 - accuracy: 0.8057
Epoch 10/10
1000/1000 [==============================] - 7s 7ms/step - loss: 0.400